***
## Сортировка через класс Meta в модели

Если сортировать объекты модели планируется одинаково при всех запросах — порядок сортировки можно объявить прямо в модели:

```py
# <название_приложения>/models.py

class <Модель>(models.Model):
    ...

    class Meta:
        ordering = ('<название_поля>',) 
```

Любой запрос к этой модели вернёт QuerySet, отсортированный по правилу, указанному в классе `Meta`. Код из листинга отсортирует QuerySet по возрастанию: от меньших к большим значениям, если поле содержит числа; по алфавиту от A до Z и от А до Я — если поле содержит строки.

При таких настройках модели в SQL-запросы будет добавлена инструкция `ORDER BY`:

```sql
ORDER BY "<таблица>"."<название_поля>" ASC 
```

Чтобы поменять порядок сортировки и упорядочить объекты от больших значений к меньшим — достаточно поставить символ «минус» `-` перед названием поля, по которому проводится сортировка:

```py
ordering = ('-<название_поля>',) 
# SQL-запрос: ORDER BY <название_поля> DESC 
```

Можно отсортировать и по нескольким полям:

```py
ordering = ('<название_поля_1>', '<название_поля_2>')  
```

***
## Метод .order_by()

Порядок сортировки можно указать и в конкретном запросе; для этого у `objects` есть метод  `.order_by()`:

```py
<Модель>.objects.order_by('<название_поля>') 
```

Если правила сортировки указаны одновременно в `Meta` и в `objects.order_by()` — будут применены правила из `objects.order_by()`. Такое «переопределение» сортировки применяют, если в большинстве случаев при работе с моделью нужен один принцип сортировки (его можно указать в `Meta`), а в каком-то частном случае требуется иная сортировка (её указывают в `objects.order_by()`). 

Сортировку, установленную в `Meta`, в каких-то запросах можно и вовсе отключить — для этого нужно вызвать метод `objects.order_by()` без параметров. Сортировка увеличивает время запроса, и если в каком-то случае она не требуется — есть смысл её отключить.

Метод `.order_by()` может сортировать и по нескольким полям:

```py
<Модель>.objects.order_by('<название_поля_1>', '<название_поля_2>') 
```

В этом случае сортировка будет производиться по первому полю, а если окажется, что у каких-то объектов значения этого поля равны — сортировка будет выполнена по второму полю.

***
## Сортировка на практике. Фильтрация по двум полям

Отсортируем результаты запроса к модели `Category` из проекта «Анфиса для друзей». Для начала освежим в памяти саму модель:

```py
# ice_cream/models.py
 
...

class Category(PublishedModel):
    title = models.CharField(max_length=256, verbose_name='Название')
    slug = models.SlugField(max_length=64, unique=True, verbose_name='Слаг')
    output_order = models.PositiveSmallIntegerField(
        default=100,
        verbose_name='Порядок отображения'
    )

    class Meta:
        verbose_name = 'категория'
        verbose_name_plural = 'Категории'
    
    def __str__(self):
        return self.title

... 
```

Ради эксперимента временно изменим view-функцию и шаблон главной страницы проекта.



При создании запроса зададим сортировку по полям `output_order` и `title`:

```py
# homepage/views.py

from django.shortcuts import render

from ice_cream.models import Category, IceCream

def index(request):
    template_name = 'homepage/index.html'
    # ice_cream_list = (
    #     IceCream.objects.values('id', 'title', 'description')
    #     .filter(is_on_main=True)
    # )
    categories = Category.objects.values(
        'id', 'output_order', 'title'
    ).order_by(
        # Сортируем записи по значению поля output_order,
        # а если значения output_order у каких-то записей равны -
        # сортируем эти записи по названию в алфавитном порядке.
        'output_order', 'title'
    )
    context = {
        # 'ice_cream_list': ice_cream_list,
        'categories': categories
    }
    return render(request, template_name, context) 
```

Будет отправлен такой SQL-запрос:

```sql
SELECT "ice_cream_category"."id",
       "ice_cream_category"."output_order",
       "ice_cream_category"."title"
FROM "ice_cream_category"
ORDER BY "ice_cream_category"."output_order" ASC, "ice_cream_category"."title" ASC 
```



Если вывести данные в шаблон…

```html
...

{% for category in categories %}  
  <p>ID: {{ category.id }}</p>
  <p>{{ category.output_order }}</p>
  <p>{{ category.title }}</p>  
{% endfor %}

... 
```

…то результат отобразится на странице:

![alt text](https://pictures.s3.yandex.net/resources/S3.2_41_1679046322.png)


***
## Ограничение и сдвиг выборки: LIMIT, OFFSET

Ограничим число возвращаемых объектов до трёх:

```py
ice_cream_list = IceCream.objects.values(
    'id', 'title', 'description'
).filter(
    is_published=True, is_on_main=True
).order_by('title')[1:4] 
```

При создании SQL-запроса конструкция `[1:4]` преобразуется в 

```sql
...

LIMIT 3 -- Количество записей. Разница между первым и вторым значениями (4 - 1) 
OFFSET 1 -- На сколько элементов сдвинуть выборку - первое значение в срезе (1) 
```

В SQL получится такой запрос:

```sql
SELECT "ice_cream_icecream"."id",
       "ice_cream_icecream"."title",
       "ice_cream_icecream"."description"
FROM "ice_cream_icecream"
WHERE ("ice_cream_icecream"."is_on_main" 
       AND "ice_cream_icecream"."is_published")
ORDER BY "ice_cream_icecream"."title" ASC
LIMIT 3
OFFSET 1
```

А запрос через ORM вернёт три словаря, «упакованных» в QuerySet. На странице результат будет выглядеть вот так:

![alt text](https://pictures.s3.yandex.net/resources/S3.2_42_1679046397.png)
